In [ ]:
1+1

In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau
from multiprocessing import Pool, cpu_count

import importlib
import os
from tqdm.auto import tqdm

# local imports
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
PLANE = 0

In [ ]:
dosave = False
plotqual = "plane%i_" % PLANE
tqual = ""
savedir = "plots_10_30_23/%s" % plotqual
plt.rcParams.update({'font.size': 14})
plottitle = "Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]

datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

filedirs = [
#     "/pnfs/sbn/persistent/users/gputnam/calib-data/RunA/DPI2023/",
#     "/pnfs/sbn/persistent/users/gputnam/calib-data/RunA/DPFNAL2022/",
    "/pnfs/sbn/persistent/users/gputnam/calib-data/Run1/",
    "/pnfs/sbn/persistent/users/gputnam/calib-data/Run2/"
]

# mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/etauS12P0_fitwvfks_fixallBgain.df"
# mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/etauS12P0_fitwvfks_fixind0gain.df"
# mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/test-detvar/etauS12P0_fitwvfks_noise1_25x.df"
# mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/test-detvar/etauS12P0_fitwvfks_noise1_35x.df"


# mcfile = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/etausS12P0C1_mcnuphase2_1k.df"

# mcfile = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/coh-detvar-wcosmic/etausS12P0C1_noise1-35x_ind0gain0-7x.df"
# mcfile = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/coh-detvar-wcosmic/etausS12P0C1_noise1-2x_ind0gain0-8x.df"
# mcfile = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/coh-detvar-wcosmic/etausS12P0C1_noise1-35x_ind0gain1-0x.df"

# mcfile = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/coh-detvar-wcosmic/etausS12P0C1_noise1-0x_ind0gain0-8x.df"

# mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/etauS12_fitwvfks_fixallgain.df"
# mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/etauS12P1_fitwvfks_fixallBgain.df"
mcfile = "/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-mc/etauS12P0_fitwvfks_fixallBgain.df"



plttxt = [
          "Run 1 + Run 2 Data",
          "Throughgoing Cosmic Muons",
          "3 < 1-Wire-Pitch < 10 mm",          
          "Front Induction Plane"]

plttitle = ""

# drivtV_file = datadir + "driftV_w.txt"
lifetime_file = datadir + "lifetimes_runArun1run2.txt"
scaleyz_file_RUN1 = datadir + "P%i_scaleYZ_Run1.txt" % PLANE
scaleyz_file_RUN2 = datadir + "P%i_scaleYZ_Run2.txt" % PLANE

In [ ]:
scaleyz_file_RUN1

In [ ]:
from XRootD import client

def open_hdf(fname):
    if not fname.startswith("/pnfs"):
        return pd.read_hdf(fname, key="df")
    
    fname = fname.replace("/pnfs", "root://fndca1.fnal.gov:1094/pnfs/fnal.gov/usr")
    d = {}
    with client.File() as f:
        f.open(fname)
        try:
            with pd.HDFStore(
                    "data.h5",
                    mode="r",
                    driver="H5FD_CORE",
                    driver_core_backing_store=0,
                    driver_core_image=f.read()[1]
                    ) as store:
                return store["df"]
        except:
            print(fname)
            raise

In [ ]:
files = sum([[filedir + f for f in os.listdir(filedir) if f.endswith(".df") and f.startswith("etau") and
         "C1" not in f and "old" not in f and "S12" in f and "P1" not in f and "P0" in f] for filedir in filedirs], [])

In [ ]:
files

In [ ]:
import gc
gc.collect()

In [ ]:
NCHUNK = 10

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.

wbin = np.linspace(0, 1701, int(1701 / 0.3) + 1)
wlo = wbin[:-1]
whi = wbin[1:]
ws = (wlo + whi) / 2.

itpc_bins = np.linspace(0, 4, 5) - 0.5

tpcs = ["EE", "EW", "WE", "WW"]


qbins = np.linspace(100, 4000, 39*4+1)
qbin_centers = (qbins[1:] + qbins[:-1]) / 2. 

In [ ]:
def fidYZ(data, iny=20, inz=100, isMC=False):
    
    ymax = 134
    ymin = -180
    
    zmin = -900
    zmax = 900
    
    fid = (data.y > ymin + iny) & (data.y < ymax - iny)\
        & (data.z < zmax - inz) & (data.z > zmin + inz)
    
    # Bad center
#     return fid & (np.abs(data.z) < 10)
    
    # Bad flange in TPC WE
#     return fid & data.tpcWE & (data.y > -40) & (data.y < -20) & (data.z > 10)

    # Bad band in TPC EE
#     return fid & data.tpcEE & (data.y > -130) & (data.y < -120) & (data.z > -130) & (data.z < -120)

    # Good part of TPC WW
#     return fid & data.tpcWW & (data.z < -10)

    # Good part of TPC WE
#     return fid & data.tpcWE & (data.z > 10)

    if not isMC:
        # Cut out some problem regions in the detector
        fid = fid & (np.abs(data.z) > 10)

#         # TPC EW
#         bad_tpcEW = data.tpcEW & (data.z > 700) & (data.y < 0)

        # TPC WW
        bad_tpcWW = data.tpcWW & (data.y > 80) & (data.z > 0)

        fid = fid & ~bad_tpcWW
    
    return fid

In [ ]:
def process_file(inp, isMC=False):
    i, f = inp

    runname = f.split("/")[7]
    scaleyz_file = scaleyz_file_RUN2 if runname == "Run2" else scaleyz_file_RUN1

    if isMC:
        print("loading!")
    df = open_hdf(f)
    
    for c in df.columns:
        if c.startswith("true"):
            del df[c]    
    if isMC:
        print("loaded!")
    
    # remove the first and last row in each group -- these may not have the full 10 hits
    df["ihit"] = df.groupby(level=0).cumcount()
    df["maxihit"] = df.ihit.groupby(level=0).max()
    df = df[(df.ihit != 0) & (df.ihit != df.maxihit)]
    
    df["tpcEE"] = df.tpcE & (df.cryostat == 0)
    df["tpcEW"] = df.tpcW & (df.cryostat == 0)
    df["tpcWE"] = df.tpcE & (df.cryostat == 1)
    df["tpcWW"] = df.tpcW & (df.cryostat == 1)

    df["dqdx"] = df.charge / df.pitch
    df["thitp"] = (df.time * tick_period - df.pandora_t0 - tanode*tick_period) / 1000.
    df["thxw"] = np.abs(np.arctan(df.dirx*df.pitch/(0.3*NCHUNK))*180/np.pi)

    if "bnb" in f:
        df["bnb"] = True
    else:
        df["bnb"] = False
        
    df["ifile"] = i

    df["lifetime"] = np.nan

    fileruns = df.run.unique()
    if not isMC:
        with open(lifetime_file) as f:
            next(f)
            for line in f:
                dat = list(map(float, line.rstrip("\n").split(" ")))
                run = int(dat[0])
                
                if run not in fileruns: continue
                
                df.loc[(df.run == run) & df.tpcEE, "lifetime"] = dat[1]
                df.loc[(df.run == run) & df.tpcEW, "lifetime"] = dat[2]
                df.loc[(df.run == run) & df.tpcWE, "lifetime"] = dat[3]
                df.loc[(df.run == run) & df.tpcWW, "lifetime"] = dat[4]
    else:
        df.lifetime = 3e3
        
    # print("Made lifetimes!")

    df["itpc"] = -1
    df.loc[df.tpcEE, "itpc"] = 0
    df.loc[df.tpcEW, "itpc"] = 1
    df.loc[df.tpcWE, "itpc"] = 2
    df.loc[df.tpcWW, "itpc"] = 3

    df["scale_yz"] = np.nan

    if not isMC:

        df["ybin"] = np.searchsorted(yz_ybin, df.y.values) - 1
        df["zbin"] = np.searchsorted(yz_zbin, df.z.values) - 1

        idx = []
        scales = []

        with open(scaleyz_file) as f:
            next(f)
            for line in f:
                dat = line.rstrip("\n").split("\t")
                tpc = dat[0]
                iy = int(dat[1])
                iz = int(dat[2])
                scale = float(dat[3])

                idx.append((tpcnames.index(tpc), iy, iz))
                scales.append(scale)

        scaledf = pd.DataFrame(scales, index=pd.MultiIndex.from_tuples(idx, names=["itpc", "ybin", "zbin"]), 
                               columns=["scale_yz"])
        dtmp = pd.merge(df[["itpc", "ybin", "zbin"]], scaledf, on=["itpc", "ybin", "zbin"], how="left")

        df["scale_yz"] = dtmp.scale_yz.values

        del dtmp

    else:
        df.scale_yz = 1.25

    BASELINE_OFFSET_ADC = 25
    baseline_corr = df.width*NCHUNK*BASELINE_OFFSET_ADC/df.pitch if isMC else 0
        
    df["dqdx_normt"] = df.dqdx * np.exp(df.thitp / df.lifetime) + baseline_corr
    df["dqdx_normwt"] = df.dqdx_normt / df.scale_yz
    when = (df.pitch > 0.3*NCHUNK) & (df.pitch < 1*NCHUNK) &\
        (df.thitp > 50) & (df.thitp < 900) &\
        (df.time > 500) & (df.time < 3000) &\
        fidYZ(df, isMC=isMC)

#     if not isMC:
#         when = when & (df.y < -140)
    
    Ndqdx_nocorr, _ = np.histogram(df.dqdx[when], 
                                   bins=qbins)
    Ndqdx_tcorr, _ = np.histogram((df.dqdx_normt)[when], 
                                  bins=qbins)

    Ndqdx_wtcorr_thxwall, _ = np.histogram((df.dqdx_normwt)[when], 
                                   bins=qbins)
    
    Ndqdx_wtcorr_thxwlow, _ = np.histogram((df.dqdx_normwt)[when & (df.thxw > 0) & (df.thxw < 20)], 
                                   bins=qbins)
    Ndqdx_wtcorr_thxwmed, _ = np.histogram((df.dqdx_normwt)[when & (df.thxw > 20) & (df.thxw < 40)], 
                                   bins=qbins)
    Ndqdx_wtcorr_thxhi, _ = np.histogram((df.dqdx_normwt)[when & (df.thxw > 40) & (df.thxw < 60)], 
                                   bins=qbins)
    
    return Ndqdx_nocorr, Ndqdx_tcorr, Ndqdx_wtcorr_thxwall, Ndqdx_wtcorr_thxwlow, Ndqdx_wtcorr_thxwmed, Ndqdx_wtcorr_thxhi

In [ ]:
process_file((0, files[0]))

In [ ]:
len(files)

In [ ]:
Ndqdx_nocorr = []
Ndqdx_tcorr = []

Ndqdx_wtcorr_thxwall = []
Ndqdx_wtcorr_thxwlow = []
Ndqdx_wtcorr_thxwmed = []
Ndqdx_wtcorr_thxwhi = []

with Pool(processes=24) as pool:
    for dqdxs in tqdm(pool.imap_unordered(process_file, enumerate(files)), total=len(files)):
        Ndqdx_nocorr.append(dqdxs[0])
        Ndqdx_tcorr.append(dqdxs[1])
        Ndqdx_wtcorr_thxwall.append(dqdxs[2])
        Ndqdx_wtcorr_thxwlow.append(dqdxs[3])
        Ndqdx_wtcorr_thxwmed.append(dqdxs[4])
        Ndqdx_wtcorr_thxwhi.append(dqdxs[5])

In [ ]:
qbins_centers = (qbins[:1] + qbins[:-1]) /2

In [ ]:
Nnocorr = np.sum(Ndqdx_nocorr, axis=0)
Ntcorr = np.sum(Ndqdx_tcorr, axis=0)


Nwtcorr_thxwall = np.sum(Ndqdx_wtcorr_thxwall, axis=0)
Nwtcorr_thxwlow = np.sum(Ndqdx_wtcorr_thxwlow, axis=0)
Nwtcorr_thxwmed = np.sum(Ndqdx_wtcorr_thxwmed, axis=0)
Nwtcorr_thxwhi = np.sum(Ndqdx_wtcorr_thxwhi, axis=0)


In [ ]:
# mcdf = open_hdf(mcfile)

In [ ]:
Nnocorr_MC, Ntcorr_MC, Nwtcorr_thxwall_MC, Nwtcorr_thxwlow_MC, Nwtcorr_thxwmed_MC, Nwtcorr_thxwhi_MC = process_file((-1, mcfile), True)

In [ ]:
s = Nwtcorr_thxwall.sum() * (qbin_centers[1] - qbin_centers[0])

_ = plt.errorbar(qbin_centers, Nwtcorr_thxwall / s, linestyle="none", yerr=np.sqrt(Nwtcorr_thxwall)/s, 
                 marker=".", color="black", label="Data")
N,_,_ = plt.hist(qbin_centers, bins=qbins, weights=Nwtcorr_thxwall_MC, histtype="step", 
             linewidth=2, label="MC", density=True)

plt.xlim([100, 2400])
plt.text(0.45, 0.35, "\n".join(plttxt),
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Equalized %i-Wire $dQ/dx$ [ADC/cm]" % NCHUNK)
plt.ylabel("Area Normalized")
plt.title(plttitle)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.legend(ncol=2)
plt.tight_layout()
if dosave: plt.savefig(savedir + "mcdata_throughgoing_dqdx.pdf")

In [ ]:
s = Nwtcorr_thxwlow.sum() * (qbin_centers[1] - qbin_centers[0])

_ = plt.errorbar(qbin_centers, Nwtcorr_thxwlow / s, linestyle="none", yerr=np.sqrt(Nwtcorr_thxwlow)/s, 
                 marker=".", color="black", label="Data")
N,_,_ = plt.hist(qbin_centers, bins=qbins, weights=Nwtcorr_thxwlow_MC, histtype="step", 
             linewidth=2, label="MC", density=True)

plt.xlim([100, 2400])
plt.text(0.45, 0.35, "\n".join(plttxt[:-1] + ["$0^\\circ < \\theta_{xw} < 20^\\circ$"] + plttxt[-1:]),
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Equalized %i-Wire $dQ/dx$ [ADC/cm]" % NCHUNK)
plt.ylabel("Area Normalized")
plt.title(plttitle)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.legend(ncol=2)
plt.tight_layout()
if dosave: plt.savefig(savedir + "mcdata_throughgoing_dqdx_thxwlow.pdf")

In [ ]:
s = Nwtcorr_thxwmed.sum() * (qbin_centers[1] - qbin_centers[0])

_ = plt.errorbar(qbin_centers, Nwtcorr_thxwmed / s, linestyle="none", yerr=np.sqrt(Nwtcorr_thxwmed)/s, 
                 marker=".", color="black", label="Data")
N,_,_ = plt.hist(qbin_centers, bins=qbins, weights=Nwtcorr_thxwmed_MC, histtype="step", 
             linewidth=2, label="MC", density=True)

plt.xlim([100, 2400])
plt.text(0.45, 0.35, "\n".join(plttxt[:-1] + ["$20^\\circ < \\theta_{xw} < 40^\\circ$"] + plttxt[-1:]),
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Equalized %i-Wire $dQ/dx$ [ADC/cm]" % NCHUNK)
plt.ylabel("Area Normalized")
plt.title(plttitle)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.legend(ncol=2)
plt.tight_layout()
if dosave: plt.savefig(savedir + "mcdata_throughgoing_dqdx_thxwmed.pdf")

In [ ]:
s = Nwtcorr_thxwhi.sum() * (qbin_centers[1] - qbin_centers[0])

_ = plt.errorbar(qbin_centers, Nwtcorr_thxwhi / s, linestyle="none", yerr=np.sqrt(Nwtcorr_thxwhi)/s, 
                 marker=".", color="black", label="Data")
N,_,_ = plt.hist(qbin_centers, bins=qbins, weights=Nwtcorr_thxwhi_MC, histtype="step", 
             linewidth=2, label="MC", density=True)

plt.xlim([100, 2400])
plt.text(0.45, 0.35, "\n".join(plttxt[:-1] + ["$40^\\circ < \\theta_{xw} < 60^\\circ$"] + plttxt[-1:]),
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Equalized %i-Wire $dQ/dx$ [ADC/cm]" % NCHUNK)
plt.ylabel("Area Normalized")
plt.title(plttitle)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.legend(ncol=2)
plt.tight_layout()
if dosave: plt.savefig(savedir + "mcdata_throughgoing_dqdx_thxwhi.pdf")

In [ ]:
CV_scale = qbin_centers[Nwtcorr_thxwall.argmax()]/qbin_centers[Nwtcorr_thxwall_MC.argmax()]
CV_scale = 1.015
s = Nwtcorr_thxwall.sum() * (qbin_centers[1] - qbin_centers[0])

_ = plt.errorbar(qbin_centers, Nwtcorr_thxwall / s, linestyle="none", yerr=np.sqrt(Nwtcorr_thxwall)/s, 
                 marker=".", color="black", label="Data")

_ = plt.hist(qbin_centers*CV_scale, bins=qbins*CV_scale, weights=Nwtcorr_thxwall_MC, histtype="step", 
             linewidth=2, label="MC", density=True)

plt.xlim([100, 2400])
plt.text(0.45, 0.35, "\n".join(plttxt),
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Equalized %i-Wire $dQ/dx$ [ADC/cm]" % NCHUNK)
plt.ylabel("Area Normalized")
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.legend(ncol=2)
plt.tight_layout()
if dosave: plt.savefig(savedir + "mcdata_scaledgain_throughgoing_dqdx.pdf")

In [ ]:
CV_scale

In [ ]:
_ = plt.hist(qbin_centers, bins=qbins, weights=Nnocorr, histtype="step", 
             linewidth=2, label="No Correction")
_ = plt.hist(qbin_centers, bins=qbins, weights=Ntcorr, histtype="step", 
             linewidth=2, label="Drift Equalization")
_ = plt.hist(qbin_centers, bins=qbins, weights=Nwtcorr_thxwall, histtype="step", 
             linewidth=2, label="Drift + Wire Plane\n+ TPC Equalization")
plt.xlim([100, 2400])
plt.text(0.45, 0.35, "\n".join(plttxt),
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("10-Wire $dQ/dx$ [ADC/cm]")
plt.ylabel("# Depositions")
plt.legend()
plt.tight_layout()
if dosave: plt.savefig(savedir + "throughgoing_dqdx.pdf")

In [ ]:
df0 = open_hdf(files[3])

In [ ]:
mcdf = open_hdf(mcfile)

In [ ]:
df0["thitp"] = (df0.time * tick_period - df0.pandora_t0 - tanode*tick_period) / 1000.
mcdf["thitp"] = (mcdf.time * tick_period - mcdf.pandora_t0 - tanode*tick_period) / 1000.

In [ ]:
df0["tpcEE"] = df0.tpcE & (df0.cryostat == 0)
df0["tpcEW"] = df0.tpcW & (df0.cryostat == 0)
df0["tpcWE"] = df0.tpcE & (df0.cryostat == 1)
df0["tpcWW"] = df0.tpcW & (df0.cryostat == 1)

mcdf["tpcEE"] = mcdf.tpcE & (mcdf.cryostat == 0)
mcdf["tpcEW"] = mcdf.tpcW & (mcdf.cryostat == 0)
mcdf["tpcWE"] = mcdf.tpcE & (mcdf.cryostat == 1)
mcdf["tpcWW"] = mcdf.tpcW & (mcdf.cryostat == 1)

In [ ]:
df0["thxw"] = np.abs(np.arctan(df0.dirx*df0.pitch/(0.3*NCHUNK))*180/np.pi)
mcdf["thxw"] = np.abs(np.arctan(mcdf.dirx*mcdf.pitch/(0.3*NCHUNK))*180/np.pi)

In [ ]:
when_df0 = (df0.pitch > 0.3*NCHUNK) & (df0.pitch < 1*NCHUNK) &\
    (df0.thitp > 50) & (df0.thitp < 900) &\
    (df0.time > 500) & (df0.time < 3000) &\
    (df0.thxw > 0) & (df0.thxw < 20) &\
    fidYZ(df0, isMC=False)

In [ ]:
when_mcdf = (mcdf.pitch > 0.3*NCHUNK) & (mcdf.pitch < 1*NCHUNK) &\
    (mcdf.thitp > 50) & (mcdf.thitp < 900) &\
    (mcdf.time > 500) & (mcdf.time < 3000) &\
    (mcdf.thxw > 0) & (mcdf.thxw < 20) &\
    fidYZ(mcdf, isMC=False)

In [ ]:
_ = plt.hist(mcdf.pitch[when_mcdf], bins=np.linspace(0.3, 1, 8)*NCHUNK, histtype="step", density=True, label="MC", linewidth=2)
_ = plt.hist(df0.pitch[when_df0], bins=np.linspace(0.3, 1, 8)*NCHUNK, histtype="step", density=True, label="Data", linewidth=2)

plt.xlabel("10-Wire Track Pitch [cm]")
plt.ylabel("Area Normalized")

plt.legend()

In [ ]:
_ = plt.hist(mcdf.thxw[when_mcdf], bins=np.linspace(0, 90, 46), histtype="step", density=True, label="MC", linewidth=2)
_ = plt.hist(df0.thxw[when_df0], bins=np.linspace(0, 90, 46), histtype="step", density=True, label="Data", linewidth=2)

plt.xlabel("$\\theta_{xw}$ [$^\\circ$]")
plt.ylabel("Area Normalized")
plt.legend()

In [ ]:
_ = plt.hist(np.abs(mcdf.dirx[when_mcdf]), bins=np.linspace(0, 1, 21), histtype="step", density=True, label="MC", linewidth=2)
_ = plt.hist(np.abs(df0.dirx[when_df0]), bins=np.linspace(0, 1, 21), histtype="step", density=True, label="Data", linewidth=2)

plt.xlabel("$|\hat{x}|$")
plt.ylabel("Area Normalized")
plt.legend()

In [ ]:
df0_var = df0.charge / df0.sumadc
mcdf_var = mcdf.charge / mcdf.sumadc
bins = np.linspace(0, 1, 21)

Nmc,_,_ = plt.hist(np.abs(mcdf_var[when_mcdf]), bins=bins, histtype="step", density=True, label="MC", linewidth=2)
Nd0,_,_ = plt.hist(np.abs(df0_var[when_df0]), bins=bins, histtype="step", density=True, label="Data", linewidth=2)

plt.xlabel("Integral / SumADC Charge")
plt.ylabel("Area Normalized")
plt.legend(loc="upper left")
plt.title("$40^\\circ < \\theta_{xw} << 60^\\circ$")

print(Nd0[-1] / Nmc[-1] )

if dosave: 
    plt.savefig(savedir + "integral_sumadc_ratio.pdf")